In [8]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Megadepth-1500

In [9]:
results = 'megadepth1500/results/results_paper.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['budget'] = df.index.str.split(' ').str[2]
df['timestamp'] = df.index.str.split(' ').str[-1]

# filter
# df = df[df['budget'].str.contains('2048')] # comment to display all budgets 

df = df[['model',  'budget', 'inlier', 'auc_5', 'auc_10', 'timestamp']]
df.sort_values(by=['budget', 'model'], inplace=True, ascending=[True, True])
df.index = range(1, 1+len(df.index))
df

,model,budget,inlier,auc_5,auc_10,timestamp
1,aliked,2048,456,40.7,56.6,20250901_230158
2,aliked+SANDesc,2048,477,43.5,60.1,20250901_230721
3,dedode+SANDesc,2048,272,45.2,61.8,20250902_092140
4,dedode-B,2048,269,42.9,59.7,20250903_200631
5,dedode-G,2048,299,46.4,63.2,20250903_202241
6,disk,2048,457,34.7,51.4,20250901_231502
7,disk+SANDesc,2048,477,36.6,54.2,20250901_232106
8,ripe,2048,335,42.3,58.0,20250901_234255
9,ripe+SANDesc,2048,325,42.9,58.7,20250901_235149
10,superpoint,2048,210,28.9,44.9,20250901_232941


## Graz High Resolution

In [10]:
scale_factors = {'scale_1.0': '4K', 'scale_1.5': 'QHD', 'scale_2.0': 'FHD'}

results = 'graz_high_res/results/results_paper.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['resolution'] = [scale_factors[s] for s in df.index.str.split(' ').str[2]]
df['budget'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

# filter
# df = df[df['resolution'].str.contains('FHD')] # comment to display all resolutions

df_auc = df[['model', 'budget', 'resolution', 'inlier', 'auc_5', 'auc_10', 'timestamp']].copy()
df_auc.sort_values(by=['resolution', 'model'], inplace=True, ascending=[False, True])
df_auc.index = range(1, 1+len(df_auc.index))
df_auc # there should be 11 elements for FHD, and 9 for QHD and 4K

,model,budget,resolution,inlier,auc_5,auc_10,timestamp
1,aliked,2048,QHD,154,51.9,65.0,20250903_123543
2,aliked+SANDesc,2048,QHD,156,64.4,75.6,20250903_131148
3,dedode+SANDesc,2048,QHD,145,56.0,69.1,20250904_010306
4,disk,2048,QHD,134,37.9,49.7,20250903_141002
5,disk+SANDesc,2048,QHD,144,54.5,67.1,20250903_144944
6,ripe,2048,QHD,109,47.5,59.9,20250903_173400
7,ripe+SANDesc,2048,QHD,117,63.2,74.7,20250903_183641
8,superpoint,2048,QHD,115,38.9,52.5,20250903_155104
9,superpoint+SANDesc,2048,QHD,161,61.7,73.8,20250903_161913
10,aliked,2048,FHD,198,54.7,67.5,20250902_113656


### Scenes breakdown

Run this after running the benchmark, otherwise no statistics are available.

In [11]:
import json
from benchmark_utils import pose_auc

with open('graz_high_res/results/results_paper.json', 'r') as f:
    data = json.load(f)
    
keys = list(data.keys())
auc_th = 5
scale = 1 # 1, 1.5, 2

breakdown, total = {}, {}
for key in keys:
    parts = key.split(' ')
    key = f'{parts[0]}_stats_scale_{float(scale)}_{parts[4]}_{parts[6]}'
    model = parts[0]

    try:
        os.path.exists(f'graz_high_res/stats/{key}.csv')
        df = pd.read_csv(f'graz_high_res/stats/{key}.csv')
    except FileNotFoundError:
        continue

    scenes = df.scene_name.unique()
    total[model] = round(pose_auc(df.e_pose, [auc_th])[0] * 100,1)

    for scene in scenes:
        scene_df = df[df.scene_name == scene]
        if scene not in breakdown:
            breakdown[scene] = {}
        breakdown[scene][model] = round(pose_auc(scene_df.e_pose, [auc_th])[0] * 100,1)

breakdown['total'] = total

In [12]:
df_breakdown = pd.DataFrame.from_dict(breakdown, orient='index').T
df_breakdown

,graz_main_square,graz_castle,graz_townhall,graz_university,graz_church,graz_clocktower,total
aliked,42.3,22.2,40.0,50.1,50.8,43.6,43.6
aliked+SANDesc,67.7,32.8,52.2,57.8,67.0,71.6,61.4
disk,31.0,3.2,39.4,47.2,52.0,18.8,32.8
disk+SANDesc,46.5,8.5,49.6,57.3,60.3,48.2,48.8
superpoint,28.6,19.1,25.5,28.1,29.5,43.7,32.3
superpoint+SANDesc,57.4,35.3,49.6,52.0,61.0,68.0,57.3
ripe,29.2,12.0,32.0,38.7,46.2,29.6,33.3
ripe+SANDesc,56.1,31.2,52.7,56.8,62.2,63.1,56.9
dedode+SANDesc,50.4,15.2,42.1,49.6,61.8,62.5,52.0
